In [6]:
import sys
sys.path.append('../src')

import torch
from torch import nn

import torchvision
import torchvision.transforms as transforms

import matplotlib.pyplot as plt
from torchinfo import summary
from tqdm.auto import tqdm
from timeit import default_timer as timer


from helper_functions import train_step, test_step, accuracy_fn, print_train_time

device = 'cuda' if torch.cuda.is_available() else 'cpu'

In [8]:
seed = 213
torch.manual_seed(seed)
torch.cuda.manual_seed(seed)

In [9]:
class BaselineModel(nn.Module):
    """
    Model architecture copying TinyVGG from: 
    https://poloclub.github.io/cnn-explainer/
    """
    def __init__(self, input_shape: int, hidden_units: int, output_shape: int):
        super().__init__()
        self.block_1 = nn.Sequential(
            nn.Conv2d(in_channels=input_shape, 
                      out_channels=hidden_units, 
                      kernel_size=3,
                      stride=1,
                      padding=1), 
            nn.ReLU(),
            nn.Conv2d(in_channels=hidden_units, 
                      out_channels=hidden_units,
                      kernel_size=3,
                      stride=1,
                      padding=1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2,
                         stride=2) 
        )
        self.block_2 = nn.Sequential(
            nn.Conv2d(hidden_units, hidden_units, 3, padding=1),
            nn.ReLU(),
            nn.Conv2d(hidden_units, hidden_units, 3, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(2)
        )
        self.classifier = nn.Sequential(
            nn.Flatten(),
            nn.Linear(in_features=hidden_units*8*8, 
                      out_features=output_shape)
        )
    
    def forward(self, x: torch.Tensor):
        x = self.block_1(x)
        x = self.block_2(x)
        x = self.classifier(x)
        return x

In [ ]:
def run_model(cinic_train, cinic_test, model, loss_fn, optimizer, epochs=3, silent=True):
    time_start = timer()
    train_loss_list, train_acc_list = [], []
    test_loss_list, test_acc_list = [], []
    # Train and test model 
    for epoch in tqdm(range(epochs)):
        train_loss, train_acc = train_step(data_loader=cinic_train, 
            model=model, 
            loss_fn=loss_fn,
            optimizer=optimizer,
            accuracy_fn=accuracy_fn,
            device=device,
            silent=silent
        )
        test_loss, test_acc = test_step(data_loader=cinic_test,
            model=model,
            loss_fn=loss_fn,
            accuracy_fn=accuracy_fn,
            silent=silent
        )

        train_loss_list.append(train_loss)
        train_acc_list.append(train_acc)
        test_loss_list.append(test_loss)
        test_acc_list.append(test_acc)

    time_end = timer()
    total_time = print_train_time(start=time_start,
                                  end=time_end,
                                  device=device)
    
    return train_loss_list, train_acc_list, test_loss_list, test_acc_list, total_time

### What I want to achieve in this notebook

train for 20 epochs 10 reruns 4 models how stable they are and if the model will overfit 


but first do it once, and plot results

In [11]:
cinic_directory = '../data'
cinic_train = torch.utils.data.DataLoader(
    torchvision.datasets.ImageFolder(cinic_directory + '/train',
    	transform=transforms.ToTensor()),
    batch_size=64, shuffle=True, num_workers=6, pin_memory=True, persistent_workers=True)

cinic_test = torch.utils.data.DataLoader(
    torchvision.datasets.ImageFolder(cinic_directory + '/test',
    	transform=transforms.ToTensor()),
    batch_size=64, shuffle=True, num_workers=6, pin_memory=True, persistent_workers=True)


model = BaselineModel(input_shape=3, 
    hidden_units=10, 
    output_shape=10).to(device)

loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(params=model.parameters(), lr=0.1)

run_model(cinic_train, cinic_test, model, loss_fn, optimizer, epochs=5)

  0%|          | 0/5 [00:00<?, ?it/s]

Epoch: 0
---------
Train loss: 2.00389 | Train accuracy: 25.87%
Test loss: 2.62001 | Test accuracy: 23.01%

Epoch: 1
---------
Train loss: 1.65986 | Train accuracy: 39.62%
Test loss: 1.95114 | Test accuracy: 32.45%

Epoch: 2
---------
Train loss: 1.55056 | Train accuracy: 43.39%
Test loss: 1.84301 | Test accuracy: 36.00%

Epoch: 3
---------
Train loss: 1.48332 | Train accuracy: 46.08%
Test loss: 1.51531 | Test accuracy: 44.89%

Epoch: 4
---------
Train loss: 1.44199 | Train accuracy: 47.73%
Test loss: 1.80364 | Test accuracy: 37.94%


Train time on cuda: 20.844 seconds
